In [31]:
# Import Libraries
import pandas as pd
import numpy as np

import pycaret

from platform import python_version
print(f'Pycaret version: {pycaret.__version__}\n'
      f'Python version: {python_version()}\n'
      f'Pandas version: {pd.__version__}\n'
      f'Numpy version: {np.__version__}')

Pycaret version: 3.0.0
Python version: 3.9.16
Pandas version: 1.5.3
Numpy version:1.23.5


In [3]:
# Loading Data
df = pd.read_csv('./data/Sleep_Efficiency.csv')
print(df.shape)
df.head()

(452, 15)


,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
0,1,65,Female,2021-03-06 01:00:00,2021-03-06 07:00:00,6.0,0.88,18,70,12,0.0,0.0,0.0,Yes,3.0
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0


In [4]:
# Drop NA and to the data without NA to CSV for future use.
print(f'Total of rows: {df.dropna().shape[0]}')
df = df.dropna()
df.to_csv('./data/Sleep_Efficiency_noNA.csv', index=False)

Total of rows: 388


In [5]:
# Separate unseen data and save it to its own CSV.
df_unseen = df.sample(n=50, random_state=42)
df = df.drop(df_unseen.index)
print(f'Data for model: {df.shape}\nData for unseen predictions: {df_unseen.shape}')
df_unseen.to_csv('./data/Sleep_Efficiency_unseen.csv')

Data for model: (338, 15)
Data for unseen predictions: (50, 15)


In [7]:
print(df.shape)
df.head()

(338, 15)


,ID,Age,Gender,Bedtime,Wakeup time,Sleep duration,Sleep efficiency,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Smoking status,Exercise frequency
1,2,69,Male,2021-12-05 02:00:00,2021-12-05 09:00:00,7.0,0.66,19,28,53,3.0,0.0,3.0,Yes,3.0
2,3,40,Female,2021-05-25 21:30:00,2021-05-25 05:30:00,8.0,0.89,20,70,10,1.0,0.0,0.0,No,3.0
3,4,40,Female,2021-11-03 02:30:00,2021-11-03 08:30:00,6.0,0.51,23,25,52,3.0,50.0,5.0,Yes,1.0
4,5,57,Male,2021-03-13 01:00:00,2021-03-13 09:00:00,8.0,0.76,27,55,18,3.0,0.0,3.0,No,3.0
6,7,27,Female,2021-07-21 21:00:00,2021-07-21 03:00:00,6.0,0.54,28,25,47,2.0,50.0,0.0,Yes,1.0


### Pycaret

In [8]:
#  Initialize the Traning environment
from pycaret.regression import *
s = setup(df, target = 'Sleep efficiency', normalize=True, ignore_features=['ID', 'Bedtime', 'Wakeup time']);

,Description,Value
0,Session id,4444
1,Target,Sleep efficiency
2,Target type,Regression
3,Original data shape,"(338, 15)"
4,Transformed data shape,"(338, 12)"
5,Transformed train set shape,"(236, 12)"
6,Transformed test set shape,"(102, 12)"
7,Ignore features,3
8,Ordinal features,2
9,Numeric features,9


In [ ]:
compare_models()

In [9]:
best = compare_models(include=['et','rf','gbr'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.0375,0.0024,0.0479,0.8438,0.0267,0.0488,1.3470
gbr,Gradient Boosting Regressor,0.0375,0.0024,0.0487,0.8404,0.0272,0.0490,1.1610
et,Extra Trees Regressor,0.0389,0.0025,0.0498,0.8345,0.0275,0.0496,1.6660


Processing:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
%%time
et = create_model('et')
t_et = tune_model(et)
f_et = finalize_model(t_et)
f_et

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0356,0.0023,0.0477,0.8782,0.0268,0.0466
1,0.0313,0.0020,0.0443,0.8540,0.0244,0.0392
2,0.0306,0.0018,0.0422,0.9153,0.0241,0.0429
3,0.0388,0.0028,0.0525,0.8105,0.0286,0.0480
4,0.0393,0.0028,0.0529,0.8127,0.0293,0.0504
5,0.0327,0.0015,0.0390,0.9427,0.0214,0.0416
6,0.0384,0.0022,0.0471,0.8797,0.0257,0.0469
7,0.0496,0.0033,0.0572,0.8352,0.0316,0.0639
8,0.0526,0.0048,0.0695,0.5248,0.0384,0.0671


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0379,0.0026,0.0512,0.8598,0.0289,0.0514
1,0.0253,0.0014,0.0380,0.8926,0.0211,0.0328
2,0.0319,0.0017,0.0414,0.9183,0.0234,0.0439
3,0.0384,0.0026,0.0511,0.8203,0.0286,0.0499
4,0.0416,0.0029,0.0537,0.8067,0.0301,0.0553
5,0.0340,0.0017,0.0413,0.9358,0.0230,0.0445
6,0.0360,0.0020,0.0445,0.8925,0.0252,0.0456
7,0.0392,0.0021,0.0453,0.8966,0.0257,0.0525
8,0.0518,0.0052,0.0723,0.4866,0.0400,0.0673


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Wall time: 2min 56s


Pipeline(memory=FastMemory(location=C:\Users\heinr\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Age', 'Sleep duration',
                                             'REM sleep percentage',
                                             'Deep sleep percentage',
                                             'Light sleep percentage',
                                             'Awakenings',
                                             'Caffeine consumption',
                                             'Alcohol consumption',
                                             'Exercise frequency'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer...
dtype: int64}]))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 ExtraTreesRegressor(bootstrap=True, criterion='absolute_error',
                                     max_depth=11, min_impurity_decrease=0.0002,
                                     min_samples_leaf=3, min_samples_split=7,
                                     n_estimators=240, n_jobs=-1,
                                     random_state=4444))])

In [11]:
%%time
rf = create_model('rf')
t_rf = tune_model(rf)
f_rf = finalize_model(t_rf)
f_rf

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0351,0.0021,0.0454,0.8899,0.0259,0.0480
1,0.0321,0.0020,0.0451,0.8486,0.0248,0.0405
2,0.0290,0.0015,0.0389,0.9277,0.0223,0.0407
3,0.0405,0.0028,0.0524,0.8109,0.0292,0.0516
4,0.0477,0.0043,0.0653,0.7138,0.0360,0.0618
5,0.0311,0.0015,0.0392,0.9419,0.0215,0.0400
6,0.0337,0.0016,0.0405,0.9111,0.0227,0.0427
7,0.0393,0.0020,0.0451,0.8972,0.0258,0.0536
8,0.0498,0.0041,0.0643,0.5937,0.0356,0.0631


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0378,0.0025,0.0498,0.8674,0.0285,0.0525
1,0.0324,0.0022,0.0466,0.8384,0.0257,0.0414
2,0.0315,0.0016,0.0405,0.9216,0.0231,0.0438
3,0.0389,0.0026,0.0509,0.8216,0.0283,0.0497
4,0.0480,0.0041,0.0642,0.7238,0.0357,0.0637
5,0.0299,0.0014,0.0372,0.9479,0.0204,0.0382
6,0.0346,0.0018,0.0420,0.9045,0.0237,0.0440
7,0.0351,0.0016,0.0405,0.9172,0.0235,0.0488
8,0.0476,0.0045,0.0670,0.5587,0.0372,0.0619


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Wall time: 2min 54s


Pipeline(memory=FastMemory(location=C:\Users\heinr\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Age', 'Sleep duration',
                                             'REM sleep percentage',
                                             'Deep sleep percentage',
                                             'Light sleep percentage',
                                             'Awakenings',
                                             'Caffeine consumption',
                                             'Alcohol consumption',
                                             'Exercise frequency'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer...
                                                                         'data_type': dtype('O'),
                                                                         'mapping': Female    0
Male      1
NaN      -1
dtype: int64},
                                                                        {'col': 'Smoking '
                                                                                'status',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': No     0
Yes    1
NaN   -1
dtype: int64}]))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 RandomForestRegressor(n_jobs=-1, random_state=4444))])

In [13]:
%%time
gbr = create_model('gbr')
t_gbr = tune_model(gbr)
f_gbr = finalize_model(t_gbr)
f_gbr

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0357,0.0023,0.0476,0.8791,0.0269,0.0487
1,0.0310,0.0019,0.0441,0.8551,0.0243,0.0395
2,0.0338,0.0019,0.0440,0.9075,0.0256,0.0479
3,0.0399,0.0028,0.0528,0.8083,0.0292,0.0504
4,0.0400,0.0031,0.0560,0.7896,0.0311,0.0518
5,0.0299,0.0014,0.0373,0.9476,0.0206,0.0393
6,0.0371,0.0019,0.0435,0.8971,0.0245,0.0470
7,0.0384,0.0023,0.0484,0.8818,0.0276,0.0519
8,0.0510,0.0047,0.0683,0.5407,0.0378,0.0652


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0394,0.0025,0.0503,0.8645,0.0291,0.0553
1,0.0331,0.0020,0.0442,0.8544,0.0245,0.0421
2,0.0368,0.0023,0.0481,0.8898,0.0278,0.0521
3,0.0397,0.0026,0.0506,0.8240,0.0278,0.0495
4,0.0485,0.0040,0.0633,0.7311,0.0359,0.0654
5,0.0307,0.0015,0.0385,0.9441,0.0210,0.0384
6,0.0368,0.0022,0.0469,0.8804,0.0265,0.0460
7,0.0394,0.0022,0.0474,0.8866,0.0274,0.0544
8,0.0545,0.0053,0.0728,0.4793,0.0402,0.0697


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
Wall time: 2min 54s


Pipeline(memory=FastMemory(location=C:\Users\heinr\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Age', 'Sleep duration',
                                             'REM sleep percentage',
                                             'Deep sleep percentage',
                                             'Light sleep percentage',
                                             'Awakenings',
                                             'Caffeine consumption',
                                             'Alcohol consumption',
                                             'Exercise frequency'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer...
                                                                         'data_type': dtype('O'),
                                                                         'mapping': Female    0
Male      1
NaN      -1
dtype: int64},
                                                                        {'col': 'Smoking '
                                                                                'status',
                                                                         'data_type': dtype('O'),
                                                                         'mapping': No     0
Yes    1
NaN   -1
dtype: int64}]))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 GradientBoostingRegressor(random_state=4444))])

In [ ]:
df.head()

In [14]:
X = df.drop(['ID','Exercise frequency'], axis=1)
y = df['Exercise frequency']

In [35]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, RandomForestRegressor

from xgboost import XGBRegressor
import lightgbm as lgb

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True, random_state=42)

models = [
    ExtraTreesRegressor(bootstrap=True, criterion='absolute_error', max_depth=11,
                    min_impurity_decrease=0.0002, min_samples_leaf=3,
                    min_samples_split=7, n_estimators=240, n_jobs=-1),
    RandomForestRegressor(n_jobs=-1),
    GradientBoostingRegressor()
]


def score_model(X, y, estimator, **kwargs):
    """
    Test various estimators.
    """
    model = Pipeline([
        ('one_hot_encoder', OneHotEncoder()),
        ('estimator', estimator)
    ])

    # Instantiate the classification model and visualizer
    model.fit(X, y, **kwargs)

    expected  = y
    predicted = model.predict(X)

    # Compute and return F1 (harmonic mean of precision and recall)
    print(f"{estimator.__class__.__name__}:\n R_sq: {r2_score(expected, predicted):.4}\n"
          f" MAE : {mean_absolute_error(expected, predicted):,.2f}\n"
         )

for model in models:
    score_model(X, y, model)

ExtraTreesRegressor:
 R_sq: 0.7136
 MAE : 0.52

RandomForestRegressor:
 R_sq: 0.928
 MAE : 0.26

GradientBoostingRegressor:
 R_sq: 0.7631
 MAE : 0.60

